# Exerimental Design with Retail Data

## Table of Contents:
* [About the Dataset](#introduction)
* [Proposal](#proposal)
* [Examining Existing Sales Call and Email Performance Rates](#existing)
* [Testing Methodology](#methodology)
    - [Sample Selection and Data Collection](#sample)
    - [Assumptions](#assumptions)
    - [What Does Success Look Like?](#look_like)
* [Initial Test Results ](#test)

 ## About the Dataset <a name="introduction"></a>
 <br>
 
This sample data set was originally uploaded by Swayanjeet Mishra as a Kaggle test dataset. The focus of the dataset is a company that sells grocery products wholesale to other suppliers. I chose this dataset for the experimental design capstone because it contains datapoints for just over 42,000 customers with information on whether they made a purchase after being contacted by a salesperson vs. contacted through a marketing email.

All sales data is for the U.S., and custoemrs are broken down by state, so we can evaluate the effects of the two marketing approaches by region as well as more broadly. For the sake of this experimental design, I refer to the company as BevCo because they mostly sell beverages.

## Proposal <a name='proposal'></a>
<br>

In the past, BevCo has maintained a large salesforce and used sales calls as the first contact for potential customers. This practice has been effective but expensive and hard to scale. In recent years, BevCo has tested email marketing and had some success. Current marketing efforts are divide rougly evenly, with half of initial outreach made through email (n = 21901) and half through direct calls (n = 20584).

BevCo is anxious to cut cost by reducing sales calls if they can prove that email marketing is at least as effective as their older, more labor intensive apprach. They have asked Marketing to create a new email campaign to test whether customer conversion rate can be maintained without the influence of direct sales calls.

## Examining Existing Sales Call vs. Email Performance <a name='existing'></a>
<br>

Before the new email campaign is implemented, it is important to research performance of the existing campaign and how it compares to sales call performance. The following data looks at both categories for all sales, and then breaks down the sales by states to see if particular sales regions have patterns that differ from the country as a whole.


In [1]:
import pandas as pd
import requests
import io


url = 'https://raw.githubusercontent.com/colettegabriel/capstone_2/master/Powersell%20DATA.csv?token=AFQPQ3ZKRKUMNDPVOXRCB5K46WKY4'

data_file = requests.get(url).content
df = pd.read_csv(io.StringIO(data_file.decode('ISO-8859-1')))
pd.options.mode.chained_assignment = None
df.drop(['Brand', 
         'Customer Size By Number of Employees',
         'Competitor Type',
         'Anonymous Variable 1',
         'Anonymous Variable 2', 
         'Anonymous Variable 3',
         'Anonymous Variable 4'], axis=1, inplace=True)

In [2]:
def cat_filter(df, category, filter):
    cat_filter = df.loc[df[category]== filter]
    return cat_filter

def state_count(df, filter1, filter2):
    yes_count = df[(df['First Contact Type'] == filter1) &
               (df['State'] == filter2) & 
               (df['Sale'] == 'Yes')].count()
    yes_count = yes_count['Sale']
    no_count = df[(df['First Contact Type'] == filter1) &
               (df['State'] == filter2) & 
               (df['Sale'] == 'No')].count()
    no_count = no_count['Sale']
    return yes_count, no_count

In [118]:
df.loc[df['Sale'] == 'Yes', 'Sale_num'] = 1
df.loc[df['Sale'] == 'No', 'Sale_num'] = 0

call_count = len(df[df['First Contact Type'] == 'Direct Contact by Salesperson'])
email_count = len(df[df['First Contact Type'] == 'Email'])

calls = cat_filter(df, 'First Contact Type', 'Direct Contact by Salesperson')
call_sale = len(calls[calls['Sale'] == 'Yes'])
call_percent = (call_sale/call_count)*100

emails = cat_filter(df, 'First Contact Type', 'Email')
email_sale = len(emails[emails['Sale'] == 'Yes'])
email_percent = (email_sale/email_count)*100

calls_reg = calls.groupby(['State']).count().reset_index()
calls_region = calls_reg[['State','Type']]
calls_region.columns = ['State', 'Call Count']

emails_reg = emails.groupby(['State']).count().reset_index()
emails_region = emails_reg[['State','Type']]
emails_region.columns = ['State', 'Call Count']


print('The current success rate for direct contact with a sales person is {} percent.'.format(round(call_percent, 2)))
print('Current success rate for email marketing is {} percent.'.format(round(email_percent, 2)))

print('\nWe also want to look at success rates of calls vs email for each sales region.')
print('First, here is a count of the number of each typeo of contact, broken down by state.')
print('This ensures that we have a high enough volumne for each state to make statistical inferences.')
print('\nSales Calls\n{}'.format(calls_region))
print('\nEmail Contact\n{}'.format(emails_region))

The current success rate for direct contact with a sales person is 27.57 percent.
Current success rate for email marketing is 18.39 percent.

We also want to look at success rates of calls vs email for each sales region.
First, here is a count of the number of each typeo of contact, broken down by state.
This ensures that we have a high enough volumne for each state to make statistical inferences.

Sales Calls
        State  Call Count
0     Arizona        2481
1  California        3031
2     Montana        1996
3      Oregon        2395
4       Texas        2216
5        Utah        5798
6  Washington        2667

Email Contact
        State  Call Count
0     Arizona        2675
1  California        4092
2     Montana        2236
3      Oregon        2030
4       Texas        2319
5        Utah        5874
6  Washington        2675


In [60]:
states = ['Arizona', 'California', 'Oregon', 'Washington', 'Texas', 'Utah', 'Montana']
call_state_yes, call_state_no = dict(), dict()
email_state_yes, email_state_no = dict(), dict()
calls_state_percent, emails_state_percent = dict(), dict()

for state in states:
    call_state_yes[state], call_state_no[state] = state_count(df, 'Direct Contact by Salesperson', state)
    email_state_yes[state], email_state_no[state] = state_count(df, 'Email', state)
    calls_state_percent[state] = (call_state_yes[state]/(call_state_yes[state]+call_state_no[state]))*100
    calls_state_percent[state] = round(calls_state_percent[state], 2)
    emails_state_percent[state] = (email_state_yes[state]/(email_state_yes[state]+email_state_no[state]))*100
    emails_state_percent[state] = round(emails_state_percent[state], 2)

In [61]:
print('Success rates for sales calls for each state are:')
for key, val in calls_state_percent.items():
    print('{:<15} {:<10}'.format(key, val))
print('\nSuccess rates for emails for each state are:')
for key, val in emails_state_percent.items():
    print('{:<15} {:<10}'.format(key, val))

Success rates for sales calls for each state are:
Arizona         23.78     
California      27.65     
Oregon          27.18     
Washington      24.48     
Texas           21.66     
Utah            33.82     
Montana         25.2      

Success rates for emails for each state are:
Arizona         20.6      
California      20.7      
Oregon          16.31     
Washington      18.84     
Texas           15.57     
Utah            17.84     
Montana         17.22     


## Testing Methodology <a name="method"></a>
<br>

BevCo currently reaches about 1,825 perspective new customers per month through ongoing email campaigns. In order to test the effectiveness of the new campaign, we propose that 20% of the contacts who would have ordinarily been designated to receive the existing email receive the new email instead.

<h3>Sample Selection and Data Collection</h3> <a name='sample'></a>

For the first month, the 20% who receive the new email campaign will be decided randomly from our list of all potential contacts in the U.S. After 2 weeks we will review the results of the campaign to get a snapshot of how the new email is performing. This will give the team the chance to check and adjust if any changes need to be made. 

If results are not statistically significant after the first month, we will continue to run the test in the same manner for an additional month to gather more data. If the test shows signs of success (i.e., being significantly more effective than the original email in achieving sales), we propose to keep the ratio at 20% during an additional month, focusing the new emails more heavily in the territories of Utah and Oregon. In these two regions sales calls outperform emails by the highest rates (16% and 11%), so they will be good places for further testing if inititial improvement is shown.

<h3>Assumptions</h3> <a name='assumptions'></a>

For the results of this test to be valid, we assume that potentional new customers receiving this new email would respond in a way similar to customers who have received the existing email in the past. We need to make this assumption because we are looking only at new customers, not repeat business.

An alternate way to test the effectiveness of the new email campaign would be to send a second series of emails to contacts who did not convert after their intial contact. To conduct this test, we would pull a sample of several thousand past contacts from the database and email them again. The division would be the same as for the new potential customers, i.e. 20% would receive the new email and the remaining 80% would receive the old email.

The good news is that both of these tests can be run simultaneously and both with provide slightly different data to ascertain the effectiveness of the new email campaign.

<h3>What Does Success Look Like?</h3> <a name="look_like"></a>

The objective is to demonstrate an improved conversion rate for the potential new customers to become buyers. The <b>success metric</b> for this campaign will be the number of emails that result in sales for the new campaign when compared to existing email sales rates. We will rate performance compared with all sales data for the past year and with sales data with Q2 from the past year. We will also break down performance of the new email campaign by sales region to compare on a state by state basis with existing data.

Secondarily we will continue compare the performance of each email campaign separately to the effectiveness of sales calls. As testing continues, we will outline additional work involving testing the email campaigns with potential customers who were previously reached through sales calls and with calling contacts who did not convert through email campaigns.

# Initial Test Results <a name="test"></a>

For the sake of this experiment, let's assume that 2 months have passed and we have gathered data for the new email campaign. 20% of the 1,825 customers contacted each month leads to a sample size of 730 for the 2 month period.

In [122]:
from scipy import stats

url = 'https://raw.githubusercontent.com/colettegabriel/capstone_2/master/Powersell%20DATA%20new%20email.csv'
    
data_file = requests.get(url).content
df_test = pd.read_csv(io.StringIO(data_file.decode('ISO-8859-1')))
pd.options.mode.chained_assignment = None
df_test.drop(['Brand', 
         'Customer Size By Number of Employees',
         'Competitor Type',
         'Anonymous Variable 1',
         'Anonymous Variable 2', 
         'Anonymous Variable 3',
         'Anonymous Variable 4'], axis=1, inplace=True)

In [123]:
df_test.loc[df_test['Sale'] == 'Yes', 'Sale_num'] = 1
df_test.loc[df_test['Sale'] == 'No', 'Sale_num'] = 0

email_count_new = len(df_test[df_test['First Contact Type'] == 'Email'])

emails_new = cat_filter(df_test, 'First Contact Type', 'Email')
email_sale_new = len(emails_new[emails_new['Sale'] == 'Yes'])
email_percent_new = (email_sale_new/email_count_new)*100

To evaluate whether the new email is statistically more effective than the old email campaign, we'll use an independent samples t-test. In this case, the null hypothesis is that the new email is not more effective than the old, in which case we continue to use the old email while the marketing team works on a new email to test.

In [134]:
print('The new campaign effective rate is {}, compared to the former rate of {}.'.format(round(email_percent_new,2), 
                                                                                        round(email_percent, 2)))
print('Is this statistically significant?\n')
print(stats.ttest_ind(emails_new['Sale_num'], emails['Sale_num']))

The new campaign effective rate is 20.82, compared to the former rate of 18.39.
Is this statistically significant?

Ttest_indResult(statistic=1.6677377276840242, pvalue=0.09538163897631992)


In this case, the p-value of 0.095 does not meet our significant criteria of 0.05. This means that while we can see that the rate of sales has increased for the new email, the amount of improvement is not statistically significant to show that the new email is more effective than the old email. Based on this result, we recommend that the marketing department devise another test email, then we will re-run this process using the revised email campaign.